In [ ]:
# NB classification with filter
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.pipeline import Pipeline

# Load the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Convert to DataFrame for ease of manipulation
df = pd.DataFrame(X, columns=iris.feature_names)
df['target'] = y

# Preprocessing: Scaling the data
scaler = StandardScaler()

# Feature selection: Using filter method with correlation
selector = SelectKBest(score_func=f_classif, k=2)

# Classifier: Naive Bayes
clf = GaussianNB()

# Create a pipeline
pipeline = Pipeline([
    ('scaler', scaler),
    ('selector', selector),
    ('classifier', clf)
])

# Cross-validation
cv = StratifiedKFold(n_splits=10)
cv_scores = cross_val_score(pipeline, X, y, cv=cv, scoring='accuracy')

# Train-test split for detailed evaluation
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Fit the pipeline on training data
pipeline.fit(X_train, y_train)

# Predict on test data
y_pred = pipeline.predict(X_test)

# Performance measures
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Print performance measures
print(f'Cross-Validation Accuracy Scores: {cv_scores}')
print(f'Mean Cross-Validation Accuracy: {np.mean(cv_scores):.2f}')
print(f'Accuracy on Test Data: {accuracy:.2f}')
print('Confusion Matrix:')
print(conf_matrix)
print('Classification Report:')
print(class_report)

# Plotting comparison graphs
# Cross-validation accuracy scores
plt.figure(figsize=(10, 5))
plt.plot(range(1, len(cv_scores) + 1), cv_scores, marker='o', linestyle='-', color='b')
plt.xlabel('Fold')
plt.ylabel('Accuracy')
plt.title('Cross-Validation Accuracy Scores')
plt.ylim(0.8, 1.0)
plt.grid()
plt.show()

# Confusion matrix heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=iris.target_names, yticklabels=iris.target_names)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
# NB classification with wrapper
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.pipeline import Pipeline

# Load the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Convert to DataFrame for ease of manipulation
df = pd.DataFrame(X, columns=iris.feature_names)
df['target'] = y

# Preprocessing: Scaling the data
scaler = StandardScaler()

# Classifier: Naive Bayes
clf = GaussianNB()

# Function to evaluate model performance for different numbers of selected features
def evaluate_feature_selection(X, y, clf, min_features, max_features):
    results = []
    for k in range(min_features, max_features + 1):
        sfs = SequentialFeatureSelector(clf, n_features_to_select=k, direction="forward", cv=StratifiedKFold(10), scoring='accuracy')
        pipeline = Pipeline([
            ('scaler', scaler),
            ('selector', sfs),
            ('classifier', clf)
        ])
        cv_scores = cross_val_score(pipeline, X, y, cv=StratifiedKFold(10), scoring='accuracy')
        results.append((k, np.mean(cv_scores)))
    return results

# Evaluate feature selection from 1 to all features
min_features = 1
max_features = X.shape[1]
results = evaluate_feature_selection(X, y, clf, min_features, max_features)

# Find the optimal number of features
optimal_num_features = max(results, key=lambda x: x[1])[0]
print(f'Optimal number of features: {optimal_num_features}')

# Plot the results
plt.figure(figsize=(10, 5))
plt.plot([r[0] for r in results], [r[1] for r in results], marker='o', linestyle='-', color='b')
plt.xlabel('Number of features selected')
plt.ylabel('Cross-validation accuracy')
plt.title('Forward Feature Selection: Number of features selected vs. Cross-validation accuracy')
plt.grid()
plt.show()

# Train-test split for detailed evaluation using the optimal number of features
sfs_optimal = SequentialFeatureSelector(clf, n_features_to_select=optimal_num_features, direction="forward", cv=StratifiedKFold(10), scoring='accuracy')
pipeline_optimal = Pipeline([
    ('scaler', scaler),
    ('selector', sfs_optimal),
    ('classifier', clf)
])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Fit the pipeline on training data
pipeline_optimal.fit(X_train, y_train)

# Predict on test data
y_pred = pipeline_optimal.predict(X_test)

# Performance measures
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Print performance measures
print(f'Cross-Validation Accuracy Scores: {[r[1] for r in results]}')
print(f'Mean Cross-Validation Accuracy with optimal features: {np.mean([r[1] for r in results if r[0] == optimal_num_features]):.2f}')
print(f'Accuracy on Test Data: {accuracy:.2f}')
print('Confusion Matrix:')
print(conf_matrix)
print('Classification Report:')
print(class_report)

# Confusion matrix heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=iris.target_names, yticklabels=iris.target_names)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
# NB classification with pca
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.pipeline import Pipeline

# Load the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Convert to DataFrame for ease of manipulation
df = pd.DataFrame(X, columns=iris.feature_names)
df['target'] = y

# Preprocessing: Scaling the data
scaler = StandardScaler()

# Feature transformation: PCA
# Function to evaluate model performance for different numbers of principal components
def evaluate_pca(X, y, clf, min_components, max_components):
    results = []
    for k in range(min_components, max_components + 1):
        pca = PCA(n_components=k)
        pipeline = Pipeline([
            ('scaler', scaler),
            ('pca', pca),
            ('classifier', clf)
        ])
        cv_scores = cross_val_score(pipeline, X, y, cv=StratifiedKFold(10), scoring='accuracy')
        results.append((k, np.mean(cv_scores)))
    return results

# Evaluate PCA transformation from 1 to all features
min_components = 1
max_components = X.shape[1]
results = evaluate_pca(X, y, GaussianNB(), min_components, max_components)

# Find the optimal number of components
optimal_num_components = max(results, key=lambda x: x[1])[0]
print(f'Optimal number of components: {optimal_num_components}')

# Plot the results
plt.figure(figsize=(10, 5))
plt.plot([r[0] for r in results], [r[1] for r in results], marker='o', linestyle='-', color='b')
plt.xlabel('Number of components selected')
plt.ylabel('Cross-validation accuracy')
plt.title('PCA: Number of components selected vs. Cross-validation accuracy')
plt.grid()
plt.show()

# Train-test split for detailed evaluation using the optimal number of components
pca_optimal = PCA(n_components=optimal_num_components)
pipeline_optimal = Pipeline([
    ('scaler', scaler),
    ('pca', pca_optimal),
    ('classifier', GaussianNB())
])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Fit the pipeline on training data
pipeline_optimal.fit(X_train, y_train)

# Predict on test data
y_pred = pipeline_optimal.predict(X_test)

# Performance measures
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Print performance measures
print(f'Cross-Validation Accuracy Scores: {[r[1] for r in results]}')
print(f'Mean Cross-Validation Accuracy with optimal components: {np.mean([r[1] for r in results if r[0] == optimal_num_components]):.2f}')
print(f'Accuracy on Test Data: {accuracy:.2f}')
print('Confusion Matrix:')
print(conf_matrix)
print('Classification Report:')
print(class_report)

# Confusion matrix heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=iris.target_names, yticklabels=iris.target_names)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
# NB classification with svd
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.pipeline import Pipeline

# Load the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Convert to DataFrame for ease of manipulation
df = pd.DataFrame(X, columns=iris.feature_names)
df['target'] = y

# Preprocessing: Scaling the data
scaler = StandardScaler()

# Classifier: Naive Bayes
clf = GaussianNB()

# Function to evaluate model performance for different numbers of components using SVD
def evaluate_svd(X, y, clf, min_components, max_components):
    results = []
    for k in range(min_components, max_components + 1):
        svd = TruncatedSVD(n_components=k)
        pipeline = Pipeline([
            ('scaler', scaler),
            ('svd', svd),
            ('classifier', clf)
        ])
        cv_scores = cross_val_score(pipeline, X, y, cv=StratifiedKFold(10), scoring='accuracy')
        results.append((k, np.mean(cv_scores)))
    return results

# Evaluate SVD transformation from 1 to all features
min_components = 1
max_components = X.shape[1]
results = evaluate_svd(X, y, clf, min_components, max_components)

# Find the optimal number of components
optimal_num_components = max(results, key=lambda x: x[1])[0]
print(f'Optimal number of components: {optimal_num_components}')

# Plot the results
plt.figure(figsize=(10, 5))
plt.plot([r[0] for r in results], [r[1] for r in results], marker='o', linestyle='-', color='b')
plt.xlabel('Number of components selected')
plt.ylabel('Cross-validation accuracy')
plt.title('SVD: Number of components selected vs. Cross-validation accuracy')
plt.grid()
plt.show()

# Train-test split for detailed evaluation using the optimal number of components
svd_optimal = TruncatedSVD(n_components=optimal_num_components)
pipeline_optimal = Pipeline([
    ('scaler', scaler),
    ('svd', svd_optimal),
    ('classifier', clf)
])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Fit the pipeline on training data
pipeline_optimal.fit(X_train, y_train)

# Predict on test data
y_pred = pipeline_optimal.predict(X_test)

# Performance measures
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Print performance measures
print(f'Cross-Validation Accuracy Scores: {[r[1] for r in results]}')
print(f'Mean Cross-Validation Accuracy with optimal components: {np.mean([r[1] for r in results if r[0] == optimal_num_components]):.2f}')
print(f'Accuracy on Test Data: {accuracy:.2f}')
print('Confusion Matrix:')
print(conf_matrix)
print('Classification Report:')
print(class_report)

# Confusion matrix heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=iris.target_names, yticklabels=iris.target_names)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
# KNN with filter
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.pipeline import Pipeline

# Load the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Convert to DataFrame for ease of manipulation
df = pd.DataFrame(X, columns=iris.feature_names)
df['target'] = y

# Preprocessing: Scaling the data
scaler = StandardScaler()

# Filter feature selection using linear coefficient (ANOVA F-test)
selector = SelectKBest(score_func=f_classif)

# Define the KNN classifier
knn = KNeighborsClassifier()

# Define the pipeline with scaling, feature selection, and KNN classifier
pipeline = Pipeline([
    ('scaler', scaler),
    ('selector', selector),
    ('knn', knn)
])

# Define the parameter grid for Grid Search
param_grid = {
    'selector__k': [1, 2, 3],  # Number of features to select
    'knn__n_neighbors': [1, 3, 5, 7, 9]  # Different values of K
}

# Perform Grid Search with cross-validation
grid_search = GridSearchCV(pipeline, param_grid, cv=StratifiedKFold(10), scoring='accuracy')
grid_search.fit(X, y)

# Get the best parameters and corresponding score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print(f'Best parameters: {best_params}')
print(f'Best cross-validation accuracy: {best_score:.2f}')

# Train-test split for detailed evaluation using the best parameters
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Update the pipeline with the best parameters
pipeline.set_params(**best_params)

# Fit the pipeline on training data
pipeline.fit(X_train, y_train)

# Predict on test data
y_pred = pipeline.predict(X_test)

# Performance measures
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Print performance measures
print(f'Accuracy on Test Data: {accuracy:.2f}')
print('Confusion Matrix:')
print(conf_matrix)
print('Classification Report:')
print(class_report)

# Confusion matrix heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=iris.target_names, yticklabels=iris.target_names)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# Plot the grid search results for KNN
results = pd.DataFrame(grid_search.cv_results_)
pivot_table = results.pivot_table(values='mean_test_score', index='param_knn__n_neighbors', columns='param_selector__k')

plt.figure(figsize=(10, 6))
sns.heatmap(pivot_table, annot=True, fmt='.3f', cmap='viridis')
plt.title('Grid Search Cross-Validation Scores')
plt.xlabel('Number of Features Selected (k)')
plt.ylabel('Number of Neighbors (n_neighbors)')
plt.show()


In [ ]:
# Knn classification with wrapper
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.pipeline import Pipeline

# Load the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Convert to DataFrame for ease of manipulation
df = pd.DataFrame(X, columns=iris.feature_names)
df['target'] = y

# Preprocessing: Scaling the data
scaler = StandardScaler()

# Define the KNN classifier
knn = KNeighborsClassifier()

# Define the forward feature selection
sfs = SequentialFeatureSelector(knn, n_features_to_select='auto', direction='forward', cv=StratifiedKFold(10))

# Define the pipeline with scaling, feature selection, and KNN classifier
pipeline = Pipeline([
    ('scaler', scaler),
    ('selector', sfs),
    ('knn', knn)
])

# Define the parameter grid for Grid Search
param_grid = {
    'knn__n_neighbors': [1, 3, 5, 7, 9]  # Different values of K
}

# Perform Grid Search with cross-validation
grid_search = GridSearchCV(pipeline, param_grid, cv=StratifiedKFold(10), scoring='accuracy')
grid_search.fit(X, y)

# Get the best parameters and corresponding score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print(f'Best parameters: {best_params}')
print(f'Best cross-validation accuracy: {best_score:.2f}')

# Train-test split for detailed evaluation using the best parameters
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Update the pipeline with the best parameters
pipeline.set_params(**best_params)

# Fit the pipeline on training data
pipeline.fit(X_train, y_train)

# Predict on test data
y_pred = pipeline.predict(X_test)

# Performance measures
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Print performance measures
print(f'Accuracy on Test Data: {accuracy:.2f}')
print('Confusion Matrix:')
print(conf_matrix)
print('Classification Report:')
print(class_report)

# Confusion matrix heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=iris.target_names, yticklabels=iris.target_names)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# Plot the grid search results for KNN
results = pd.DataFrame(grid_search.cv_results_)
plt.figure(figsize=(10, 6))
plt.plot(results['param_knn__n_neighbors'], results['mean_test_score'], marker='o')
plt.title('Grid Search Cross-Validation Scores')
plt.xlabel('Number of Neighbors (n_neighbors)')
plt.ylabel('Mean Cross-Validation Accuracy')
plt.grid()
plt.show()


In [ ]:
# KNN classification with pca
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.pipeline import Pipeline

# Load the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Convert to DataFrame for ease of manipulation
df = pd.DataFrame(X, columns=iris.feature_names)
df['target'] = y

# Preprocessing: Scaling the data
scaler = StandardScaler()

# Define the PCA transformation
pca = PCA()

# Define the KNN classifier
knn = KNeighborsClassifier()

# Define the pipeline with scaling, PCA, and KNN classifier
pipeline = Pipeline([
    ('scaler', scaler),
    ('pca', pca),
    ('knn', knn)
])

# Define the parameter grid for Grid Search
param_grid = {
    'pca__n_components': [1, 2, 3],  # Number of components for PCA
    'knn__n_neighbors': [1, 3, 5, 7, 9]  # Different values of K
}

# Perform Grid Search with cross-validation
grid_search = GridSearchCV(pipeline, param_grid, cv=StratifiedKFold(10), scoring='accuracy')
grid_search.fit(X, y)

# Get the best parameters and corresponding score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print(f'Best parameters: {best_params}')
print(f'Best cross-validation accuracy: {best_score:.2f}')

# Train-test split for detailed evaluation using the best parameters
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Update the pipeline with the best parameters
pipeline.set_params(**best_params)

# Fit the pipeline on training data
pipeline.fit(X_train, y_train)

# Predict on test data
y_pred = pipeline.predict(X_test)

# Performance measures
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Print performance measures
print(f'Accuracy on Test Data: {accuracy:.2f}')
print('Confusion Matrix:')
print(conf_matrix)
print('Classification Report:')
print(class_report)

# Confusion matrix heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=iris.target_names, yticklabels=iris.target_names)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# Plot the grid search results for KNN
results = pd.DataFrame(grid_search.cv_results_)
pivot_table = results.pivot_table(values='mean_test_score', index='param_knn__n_neighbors', columns='param_pca__n_components')

plt.figure(figsize=(10, 6))
sns.heatmap(pivot_table, annot=True, fmt='.3f', cmap='viridis')
plt.title('Grid Search Cross-Validation Scores')
plt.xlabel('Number of PCA Components')
plt.ylabel('Number of Neighbors (n_neighbors)')
plt.show()


In [ ]:
# Knn classification with svd
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.pipeline import Pipeline

# Load the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Convert to DataFrame for ease of manipulation
df = pd.DataFrame(X, columns=iris.feature_names)
df['target'] = y

# Preprocessing: Scaling the data
scaler = StandardScaler()

# Define the SVD transformation
svd = TruncatedSVD()

# Define the KNN classifier
knn = KNeighborsClassifier()

# Define the pipeline with scaling, SVD, and KNN classifier
pipeline = Pipeline([
    ('scaler', scaler),
    ('svd', svd),
    ('knn', knn)
])

# Define the parameter grid for Grid Search
param_grid = {
    'svd__n_components': [1, 2, 3],  # Number of components for SVD
    'knn__n_neighbors': [1, 3, 5, 7, 9]  # Different values of K
}

# Perform Grid Search with cross-validation
grid_search = GridSearchCV(pipeline, param_grid, cv=StratifiedKFold(10), scoring='accuracy')
grid_search.fit(X, y)

# Get the best parameters and corresponding score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print(f'Best parameters: {best_params}')
print(f'Best cross-validation accuracy: {best_score:.2f}')

# Train-test split for detailed evaluation using the best parameters
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Update the pipeline with the best parameters
pipeline.set_params(**best_params)

# Fit the pipeline on training data
pipeline.fit(X_train, y_train)

# Predict on test data
y_pred = pipeline.predict(X_test)

# Performance measures
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Print performance measures
print(f'Accuracy on Test Data: {accuracy:.2f}')
print('Confusion Matrix:')
print(conf_matrix)
print('Classification Report:')
print(class_report)

# Confusion matrix heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=iris.target_names, yticklabels=iris.target_names)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# Plot the grid search results for KNN
results = pd.DataFrame(grid_search.cv_results_)
pivot_table = results.pivot_table(values='mean_test_score', index='param_knn__n_neighbors', columns='param_svd__n_components')

plt.figure(figsize=(10, 6))
sns.heatmap(pivot_table, annot=True, fmt='.3f', cmap='viridis')
plt.title('Grid Search Cross-Validation Scores')
plt.xlabel('Number of SVD Components')
plt.ylabel('Number of Neighbors (n_neighbors)')
plt.show()


In [ ]:
# Logistic with filter
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.pipeline import Pipeline

# Load the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Convert to DataFrame for ease of manipulation
df = pd.DataFrame(X, columns=iris.feature_names)
df['target'] = y

# Preprocessing: Scaling the data
scaler = StandardScaler()

# Filter feature selection using linear coefficient (ANOVA F-test)
selector = SelectKBest(score_func=f_classif)

# Define the logistic regression classifier
logistic = LogisticRegression(max_iter=200)

# Define the pipeline with scaling, feature selection, and logistic regression classifier
pipeline = Pipeline([
    ('scaler', scaler),
    ('selector', selector),
    ('logistic', logistic)
])

# Define the parameter grid for Grid Search
param_grid = {
    'selector__k': [1, 2, 3],  # Number of features to select
    'logistic__C': [0.01, 0.1, 1, 10, 100]  # Inverse of regularization strength
}

# Perform Grid Search with cross-validation
grid_search = GridSearchCV(pipeline, param_grid, cv=StratifiedKFold(10), scoring='accuracy')
grid_search.fit(X, y)

# Get the best parameters and corresponding score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print(f'Best parameters: {best_params}')
print(f'Best cross-validation accuracy: {best_score:.2f}')

# Train-test split for detailed evaluation using the best parameters
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Update the pipeline with the best parameters
pipeline.set_params(**best_params)

# Fit the pipeline on training data
pipeline.fit(X_train, y_train)

# Predict on test data
y_pred = pipeline.predict(X_test)

# Performance measures
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Print performance measures
print(f'Accuracy on Test Data: {accuracy:.2f}')
print('Confusion Matrix:')
print(conf_matrix)
print('Classification Report:')
print(class_report)

# Confusion matrix heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=iris.target_names, yticklabels=iris.target_names)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# Plot the grid search results for Logistic Regression
results = pd.DataFrame(grid_search.cv_results_)
pivot_table = results.pivot_table(values='mean_test_score', index='param_logistic__C', columns='param_selector__k')

plt.figure(figsize=(10, 6))
sns.heatmap(pivot_table, annot=True, fmt='.3f', cmap='viridis')
plt.title('Grid Search Cross-Validation Scores')
plt.xlabel('Number of Selected Features (k)')
plt.ylabel('Inverse of Regularization Strength (C)')
plt.show()


In [ ]:
# Logistic with wrapper
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.pipeline import Pipeline

# Load the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Convert to DataFrame for ease of manipulation
df = pd.DataFrame(X, columns=iris.feature_names)
df['target'] = y

# Preprocessing: Scaling the data
scaler = StandardScaler()

# Define the logistic regression classifier
logistic = LogisticRegression(max_iter=200)

# Define the forward feature selection
sfs = SequentialFeatureSelector(logistic, n_features_to_select='auto', direction='forward', cv=StratifiedKFold(10))

# Define the pipeline with scaling, feature selection, and logistic regression classifier
pipeline = Pipeline([
    ('scaler', scaler),
    ('selector', sfs),
    ('logistic', logistic)
])

# Define the parameter grid for Grid Search
param_grid = {
    'selector__n_features_to_select': [1, 2, 3, 4],  # Number of features to select
    'logistic__C': [0.01, 0.1, 1, 10, 100]  # Inverse of regularization strength
}

# Perform Grid Search with cross-validation
grid_search = GridSearchCV(pipeline, param_grid, cv=StratifiedKFold(10), scoring='accuracy')
grid_search.fit(X, y)

# Get the best parameters and corresponding score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print(f'Best parameters: {best_params}')
print(f'Best cross-validation accuracy: {best_score:.2f}')

# Train-test split for detailed evaluation using the best parameters
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Update the pipeline with the best parameters
pipeline.set_params(**best_params)

# Fit the pipeline on training data
pipeline.fit(X_train, y_train)

# Predict on test data
y_pred = pipeline.predict(X_test)

# Performance measures
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Print performance measures
print(f'Accuracy on Test Data: {accuracy:.2f}')
print('Confusion Matrix:')
print(conf_matrix)
print('Classification Report:')
print(class_report)

# Confusion matrix heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=iris.target_names, yticklabels=iris.target_names)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# Plot the grid search results for Logistic Regression
results = pd.DataFrame(grid_search.cv_results_)
pivot_table = results.pivot_table(values='mean_test_score', index='param_logistic__C', columns='param_selector__n_features_to_select')

plt.figure(figsize=(10, 6))
sns.heatmap(pivot_table, annot=True, fmt='.3f', cmap='viridis')
plt.title('Grid Search Cross-Validation Scores')
plt.xlabel('Number of Selected Features')
plt.ylabel('Inverse of Regularization Strength (C)')
plt.show()


In [ ]:
# Logistic with pca
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.pipeline import Pipeline

# Load the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Convert to DataFrame for ease of manipulation
df = pd.DataFrame(X, columns=iris.feature_names)
df['target'] = y

# Preprocessing: Scaling the data
scaler = StandardScaler()

# Define the PCA transformation
pca = PCA()

# Define the logistic regression classifier
logistic = LogisticRegression(max_iter=200)

# Define the pipeline with scaling, PCA, and logistic regression classifier
pipeline = Pipeline([
    ('scaler', scaler),
    ('pca', pca),
    ('logistic', logistic)
])

# Define the parameter grid for Grid Search
param_grid = {
    'pca__n_components': [1, 2, 3, 4],  # Number of components for PCA
    'logistic__C': [0.01, 0.1, 1, 10, 100]  # Inverse of regularization strength
}

# Perform Grid Search with cross-validation
grid_search = GridSearchCV(pipeline, param_grid, cv=StratifiedKFold(10), scoring='accuracy')
grid_search.fit(X, y)

# Get the best parameters and corresponding score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print(f'Best parameters: {best_params}')
print(f'Best cross-validation accuracy: {best_score:.2f}')

# Train-test split for detailed evaluation using the best parameters
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Update the pipeline with the best parameters
pipeline.set_params(**best_params)

# Fit the pipeline on training data
pipeline.fit(X_train, y_train)

# Predict on test data
y_pred = pipeline.predict(X_test)

# Performance measures
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Print performance measures
print(f'Accuracy on Test Data: {accuracy:.2f}')
print('Confusion Matrix:')
print(conf_matrix)
print('Classification Report:')
print(class_report)

# Confusion matrix heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=iris.target_names, yticklabels=iris.target_names)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# Plot the grid search results for Logistic Regression
results = pd.DataFrame(grid_search.cv_results_)
pivot_table = results.pivot_table(values='mean_test_score', index='param_logistic__C', columns='param_pca__n_components')

plt.figure(figsize=(10, 6))
sns.heatmap(pivot_table, annot=True, fmt='.3f', cmap='viridis')
plt.title('Grid Search Cross-Validation Scores')
plt.xlabel('Number of PCA Components')
plt.ylabel('Inverse of Regularization Strength (C)')
plt.show()


In [ ]:
# Logistic with svd
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.pipeline import Pipeline

# Load the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Convert to DataFrame for ease of manipulation
df = pd.DataFrame(X, columns=iris.feature_names)
df['target'] = y

# Preprocessing: Scaling the data
scaler = StandardScaler()

# Define the SVD transformation
svd = TruncatedSVD()

# Define the logistic regression classifier
logistic = LogisticRegression(max_iter=200)

# Define the pipeline with scaling, SVD, and logistic regression classifier
pipeline = Pipeline([
    ('scaler', scaler),
    ('svd', svd),
    ('logistic', logistic)
])

# Define the parameter grid for Grid Search
param_grid = {
    'svd__n_components': [1, 2, 3, 4],  # Number of components for SVD
    'logistic__C': [0.01, 0.1, 1, 10, 100]  # Inverse of regularization strength
}

# Perform Grid Search with cross-validation
grid_search = GridSearchCV(pipeline, param_grid, cv=StratifiedKFold(10), scoring='accuracy')
grid_search.fit(X, y)

# Get the best parameters and corresponding score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print(f'Best parameters: {best_params}')
print(f'Best cross-validation accuracy: {best_score:.2f}')

# Train-test split for detailed evaluation using the best parameters
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Update the pipeline with the best parameters
pipeline.set_params(**best_params)

# Fit the pipeline on training data
pipeline.fit(X_train, y_train)

# Predict on test data
y_pred = pipeline.predict(X_test)

# Performance measures
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Print performance measures
print(f'Accuracy on Test Data: {accuracy:.2f}')
print('Confusion Matrix:')
print(conf_matrix)
print('Classification Report:')
print(class_report)

# Confusion matrix heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=iris.target_names, yticklabels=iris.target_names)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# Plot the grid search results for Logistic Regression
results = pd.DataFrame(grid_search.cv_results_)
pivot_table = results.pivot_table(values='mean_test_score', index='param_logistic__C', columns='param_svd__n_components')

plt.figure(figsize=(10, 6))
sns.heatmap(pivot_table, annot=True, fmt='.3f', cmap='viridis')
plt.title('Grid Search Cross-Validation Scores')
plt.xlabel('Number of SVD Components')
plt.ylabel('Inverse of Regularization Strength (C)')
plt.show()


In [ ]:
# NN with filter
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.pipeline import Pipeline

# Load the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Convert to DataFrame for ease of manipulation
df = pd.DataFrame(X, columns=iris.feature_names)
df['target'] = y

# Preprocessing: Scaling the data
scaler = StandardScaler()

# Filter feature selection using ANOVA F-test
selector = SelectKBest(score_func=f_classif)

# Define the neural network classifier
mlp = MLPClassifier(max_iter=200)

# Define the pipeline with scaling, feature selection, and neural network classifier
pipeline = Pipeline([
    ('scaler', scaler),
    ('selector', selector),
    ('mlp', mlp)
])

# Define the parameter grid for Grid Search
param_grid = {
    'selector__k': [1, 2, 3],  # Number of features to select
    'mlp__hidden_layer_sizes': [(100,), (50, 50), (25, 25, 25)],  # Hidden layer sizes
    'mlp__alpha': [0.0001, 0.001, 0.01]  # L2 penalty (regularization term) parameter
}

# Perform Grid Search with cross-validation
grid_search = GridSearchCV(pipeline, param_grid, cv=StratifiedKFold(10), scoring='accuracy')
grid_search.fit(X, y)

# Get the best parameters and corresponding score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print(f'Best parameters: {best_params}')
print(f'Best cross-validation accuracy: {best_score:.2f}')

# Train-test split for detailed evaluation using the best parameters
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Update the pipeline with the best parameters
pipeline.set_params(**best_params)

# Fit the pipeline on training data
pipeline.fit(X_train, y_train)

# Predict on test data
y_pred = pipeline.predict(X_test)

# Performance measures
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Print performance measures
print(f'Accuracy on Test Data: {accuracy:.2f}')
print('Confusion Matrix:')
print(conf_matrix)
print('Classification Report:')
print(class_report)

# Confusion matrix heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=iris.target_names, yticklabels=iris.target_names)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# Plot the grid search results for Neural Network
results = pd.DataFrame(grid_search.cv_results_)
pivot_table = results.pivot_table(values='mean_test_score', index='param_selector__k', columns='param_mlp__hidden_layer_sizes')

plt.figure(figsize=(10, 6))
sns.heatmap(pivot_table, annot=True, fmt='.3f', cmap='viridis')
plt.title('Grid Search Cross-Validation Scores')
plt.xlabel('Hidden Layer Sizes')
plt.ylabel('Number of Selected Features (k)')
plt.show()


In [ ]:
# NN with wrapper
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.pipeline import Pipeline

# Load the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Convert to DataFrame for ease of manipulation
df = pd.DataFrame(X, columns=iris.feature_names)
df['target'] = y

# Preprocessing: Scaling the data
scaler = StandardScaler()

# Define the neural network classifier
mlp = MLPClassifier(max_iter=200)

# Define the forward feature selection
sfs = SequentialFeatureSelector(mlp, n_features_to_select='auto', direction='forward', cv=StratifiedKFold(10))

# Define the pipeline with scaling, feature selection, and neural network classifier
pipeline = Pipeline([
    ('scaler', scaler),
    ('selector', sfs),
    ('mlp', mlp)
])

# Define the parameter grid for Grid Search
param_grid = {
    'selector__n_features_to_select': [1, 2, 3, 4],  # Number of features to select
    'mlp__hidden_layer_sizes': [(100,), (50, 50), (25, 25, 25)],  # Hidden layer sizes
    'mlp__alpha': [0.0001, 0.001, 0.01]  # L2 penalty (regularization term) parameter
}

# Perform Grid Search with cross-validation
grid_search = GridSearchCV(pipeline, param_grid, cv=StratifiedKFold(10), scoring='accuracy')
grid_search.fit(X, y)

# Get the best parameters and corresponding score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print(f'Best parameters: {best_params}')
print(f'Best cross-validation accuracy: {best_score:.2f}')

# Train-test split for detailed evaluation using the best parameters
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Update the pipeline with the best parameters
pipeline.set_params(**best_params)

# Fit the pipeline on training data
pipeline.fit(X_train, y_train)

# Predict on test data
y_pred = pipeline.predict(X_test)

# Performance measures
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Print performance measures
print(f'Accuracy on Test Data: {accuracy:.2f}')
print('Confusion Matrix:')
print(conf_matrix)
print('Classification Report:')
print(class_report)

# Confusion matrix heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=iris.target_names, yticklabels=iris.target_names)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# Plot the grid search results for Neural Network
results = pd.DataFrame(grid_search.cv_results_)
pivot_table = results.pivot_table(values='mean_test_score', index='param_mlp__hidden_layer_sizes', columns='param_selector__n_features_to_select')

plt.figure(figsize=(10, 6))
sns.heatmap(pivot_table, annot=True, fmt='.3f', cmap='viridis')
plt.title('Grid Search Cross-Validation Scores')
plt.xlabel('Number of Selected Features')
plt.ylabel('Hidden Layer Sizes')
plt.show()


In [ ]:
# NN with pca
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.pipeline import Pipeline

# Load the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Convert to DataFrame for ease of manipulation
df = pd.DataFrame(X, columns=iris.feature_names)
df['target'] = y

# Preprocessing: Scaling the data
scaler = StandardScaler()

# Define the PCA transformation
pca = PCA()

# Define the neural network classifier
mlp = MLPClassifier(max_iter=200)

# Define the pipeline with scaling, PCA, and neural network classifier
pipeline = Pipeline([
    ('scaler', scaler),
    ('pca', pca),
    ('mlp', mlp)
])

# Define the parameter grid for Grid Search
param_grid = {
    'pca__n_components': [1, 2, 3, 4],  # Number of components for PCA
    'mlp__hidden_layer_sizes': [(100,), (50, 50), (25, 25, 25)],  # Hidden layer sizes
    'mlp__alpha': [0.0001, 0.001, 0.01]  # L2 penalty (regularization term) parameter
}

# Perform Grid Search with cross-validation
grid_search = GridSearchCV(pipeline, param_grid, cv=StratifiedKFold(10), scoring='accuracy')
grid_search.fit(X, y)

# Get the best parameters and corresponding score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print(f'Best parameters: {best_params}')
print(f'Best cross-validation accuracy: {best_score:.2f}')

# Train-test split for detailed evaluation using the best parameters
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Update the pipeline with the best parameters
pipeline.set_params(**best_params)

# Fit the pipeline on training data
pipeline.fit(X_train, y_train)

# Predict on test data
y_pred = pipeline.predict(X_test)

# Performance measures
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Print performance measures
print(f'Accuracy on Test Data: {accuracy:.2f}')
print('Confusion Matrix:')
print(conf_matrix)
print('Classification Report:')
print(class_report)

# Confusion matrix heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=iris.target_names, yticklabels=iris.target_names)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# Plot the grid search results for Neural Network
results = pd.DataFrame(grid_search.cv_results_)
pivot_table = results.pivot_table(values='mean_test_score', index='param_pca__n_components', columns='param_mlp__hidden_layer_sizes')

plt.figure(figsize=(10, 6))
sns.heatmap(pivot_table, annot=True, fmt='.3f', cmap='viridis')
plt.title('Grid Search Cross-Validation Scores')
plt.xlabel('Number of PCA Components')
plt.ylabel('Hidden Layer Sizes')
plt.show()


In [ ]:
# NN with svd
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.pipeline import Pipeline

# Load the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Convert to DataFrame for ease of manipulation
df = pd.DataFrame(X, columns=iris.feature_names)
df['target'] = y

# Preprocessing: Scaling the data
scaler = StandardScaler()

# Define the SVD transformation
svd = TruncatedSVD()

# Define the neural network classifier
mlp = MLPClassifier(max_iter=200)

# Define the pipeline with scaling, SVD, and neural network classifier
pipeline = Pipeline([
    ('scaler', scaler),
    ('svd', svd),
    ('mlp', mlp)
])

# Define the parameter grid for Grid Search
param_grid = {
    'svd__n_components': [1, 2, 3, 4],  # Number of components for SVD
    'mlp__hidden_layer_sizes': [(100,), (50, 50), (25, 25, 25)],  # Hidden layer sizes
    'mlp__alpha': [0.0001, 0.001, 0.01]  # L2 penalty (regularization term) parameter
}

# Perform Grid Search with cross-validation
grid_search = GridSearchCV(pipeline, param_grid, cv=StratifiedKFold(10), scoring='accuracy')
grid_search.fit(X, y)

# Get the best parameters and corresponding score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print(f'Best parameters: {best_params}')
print(f'Best cross-validation accuracy: {best_score:.2f}')

# Train-test split for detailed evaluation using the best parameters
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Update the pipeline with the best parameters
pipeline.set_params(**best_params)

# Fit the pipeline on training data
pipeline.fit(X_train, y_train)

# Predict on test data
y_pred = pipeline.predict(X_test)

# Performance measures
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Print performance measures
print(f'Accuracy on Test Data: {accuracy:.2f}')
print('Confusion Matrix:')
print(conf_matrix)
print('Classification Report:')
print(class_report)

# Confusion matrix heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=iris.target_names, yticklabels=iris.target_names)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# Plot the grid search results for Neural Network
results = pd.DataFrame(grid_search.cv_results_)
pivot_table = results.pivot_table(values='mean_test_score', index='param_svd__n_components', columns='param_mlp__hidden_layer_sizes')

plt.figure(figsize=(10, 6))
sns.heatmap(pivot_table, annot=True, fmt='.3f', cmap='viridis')
plt.title('Grid Search Cross-Validation Scores')
plt.xlabel('Number of SVD Components')
plt.ylabel('Hidden Layer Sizes')
plt.show()


In [ ]:
# Decision tree with filter
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Load the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Convert to DataFrame for ease of manipulation
df = pd.DataFrame(X, columns=iris.feature_names)
df['target'] = y

# Preprocessing: Scaling the data
scaler = StandardScaler()

# Feature selection using linear correlation
selector = SelectKBest(score_func=f_classif, k=3)  # Select top 3 features

# Define the Decision Tree classifier
decision_tree = DecisionTreeClassifier()

# Define the pipeline with scaling, feature selection, and Decision Tree classifier
pipeline = Pipeline([
    ('scaler', scaler),
    ('selector', selector),
    ('decision_tree', decision_tree)
])

# Define the parameter grid for Grid Search
param_grid = {
    'decision_tree__max_depth': [None, 5, 10, 15],  # Maximum depth of the tree
    'decision_tree__min_samples_split': [2, 5, 10],  # Minimum number of samples required to split a node
    'decision_tree__min_samples_leaf': [1, 2, 4]  # Minimum number of samples required at each leaf node
}

# Perform Grid Search with cross-validation
grid_search = GridSearchCV(pipeline, param_grid, cv=StratifiedKFold(10), scoring='accuracy')
grid_search.fit(X, y)

# Get the best parameters and corresponding score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print(f'Best parameters: {best_params}')
print(f'Best cross-validation accuracy: {best_score:.2f}')

# Train-test split for detailed evaluation using the best parameters
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Update the pipeline with the best parameters
pipeline.set_params(**best_params)

# Fit the pipeline on training data
pipeline.fit(X_train, y_train)

# Predict on test data
y_pred = pipeline.predict(X_test)

# Performance measures
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Print performance measures
print(f'Accuracy on Test Data: {accuracy:.2f}')
print('Confusion Matrix:')
print(conf_matrix)
print('Classification Report:')
print(class_report)

# Confusion matrix heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=iris.target_names, yticklabels=iris.target_names)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
# Decision tree with wrapper
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Load the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Convert to DataFrame for ease of manipulation
df = pd.DataFrame(X, columns=iris.feature_names)
df['target'] = y

# Preprocessing: Scaling the data
scaler = StandardScaler()

# Define the Decision Tree classifier
decision_tree = DecisionTreeClassifier()

# Define the forward feature selection
sfs = SequentialFeatureSelector(decision_tree, n_features_to_select='auto', direction='forward', cv=StratifiedKFold(10))

# Define the pipeline with scaling, feature selection, and Decision Tree classifier
pipeline = Pipeline([
    ('scaler', scaler),
    ('selector', sfs),
    ('decision_tree', decision_tree)
])

# Define the parameter grid for Grid Search
param_grid = {
    'decision_tree__max_depth': [None, 5, 10, 15],  # Maximum depth of the tree
    'decision_tree__min_samples_split': [2, 5, 10],  # Minimum number of samples required to split a node
    'decision_tree__min_samples_leaf': [1, 2, 4]  # Minimum number of samples required at each leaf node
}

# Perform Grid Search with cross-validation
grid_search = GridSearchCV(pipeline, param_grid, cv=StratifiedKFold(10), scoring='accuracy')
grid_search.fit(X, y)

# Get the best parameters and corresponding score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print(f'Best parameters: {best_params}')
print(f'Best cross-validation accuracy: {best_score:.2f}')

# Train-test split for detailed evaluation using the best parameters
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Update the pipeline with the best parameters
pipeline.set_params(**best_params)

# Fit the pipeline on training data
pipeline.fit(X_train, y_train)

# Predict on test data
y_pred = pipeline.predict(X_test)

# Performance measures
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Print performance measures
print(f'Accuracy on Test Data: {accuracy:.2f}')
print('Confusion Matrix:')
print(conf_matrix)
print('Classification Report:')
print(class_report)

# Confusion matrix heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=iris.target_names, yticklabels=iris.target_names)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
# Decision tree with pca
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Load the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Convert to DataFrame for ease of manipulation
df = pd.DataFrame(X, columns=iris.feature_names)
df['target'] = y

# Preprocessing: Scaling the data
scaler = StandardScaler()

# Define the PCA transformation
pca = PCA()

# Define the Decision Tree classifier
decision_tree = DecisionTreeClassifier()

# Define the pipeline with scaling, PCA, and Decision Tree classifier
pipeline = Pipeline([
    ('scaler', scaler),
    ('pca', pca),
    ('decision_tree', decision_tree)
])

# Define the parameter grid for Grid Search
param_grid = {
    'pca__n_components': [1, 2, 3],  # Number of components for PCA
    'decision_tree__max_depth': [None, 5, 10, 15],  # Maximum depth of the tree
    'decision_tree__min_samples_split': [2, 5, 10],  # Minimum number of samples required to split a node
    'decision_tree__min_samples_leaf': [1, 2, 4]  # Minimum number of samples required at each leaf node
}

# Perform Grid Search with cross-validation
grid_search = GridSearchCV(pipeline, param_grid, cv=StratifiedKFold(10), scoring='accuracy')
grid_search.fit(X, y)

# Get the best parameters and corresponding score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print(f'Best parameters: {best_params}')
print(f'Best cross-validation accuracy: {best_score:.2f}')

# Train-test split for detailed evaluation using the best parameters
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Update the pipeline with the best parameters
pipeline.set_params(**best_params)

# Fit the pipeline on training data
pipeline.fit(X_train, y_train)

# Predict on test data
y_pred = pipeline.predict(X_test)

# Performance measures
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Print performance measures
print(f'Accuracy on Test Data: {accuracy:.2f}')
print('Confusion Matrix:')
print(conf_matrix)
print('Classification Report:')
print(class_report)

# Confusion matrix heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=iris.target_names, yticklabels=iris.target_names)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
# Decision tree with svd
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Load the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Convert to DataFrame for ease of manipulation
df = pd.DataFrame(X, columns=iris.feature_names)
df['target'] = y

# Preprocessing: Scaling the data
scaler = StandardScaler()

# Define the SVD transformation
svd = TruncatedSVD()

# Define the Decision Tree classifier
decision_tree = DecisionTreeClassifier()

# Define the pipeline with scaling, SVD, and Decision Tree classifier
pipeline = Pipeline([
    ('scaler', scaler),
    ('svd', svd),
    ('decision_tree', decision_tree)
])

# Define the parameter grid for Grid Search
param_grid = {
    'svd__n_components': [1, 2, 3],  # Number of components for SVD
    'decision_tree__max_depth': [None, 5, 10, 15],  # Maximum depth of the tree
    'decision_tree__min_samples_split': [2, 5, 10],  # Minimum number of samples required to split a node
    'decision_tree__min_samples_leaf': [1, 2, 4]  # Minimum number of samples required at each leaf node
}

# Perform Grid Search with cross-validation
grid_search = GridSearchCV(pipeline, param_grid, cv=StratifiedKFold(10), scoring='accuracy')
grid_search.fit(X, y)

# Get the best parameters and corresponding score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print(f'Best parameters: {best_params}')
print(f'Best cross-validation accuracy: {best_score:.2f}')

# Train-test split for detailed evaluation using the best parameters
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Update the pipeline with the best parameters
pipeline.set_params(**best_params)

# Fit the pipeline on training data
pipeline.fit(X_train, y_train)

# Predict on test data
y_pred = pipeline.predict(X_test)

# Performance measures
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Print performance measures
print(f'Accuracy on Test Data: {accuracy:.2f}')
print('Confusion Matrix:')
print(conf_matrix)
print('Classification Report:')
print(class_report)

# Confusion matrix heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=iris.target_names, yticklabels=iris.target_names)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
# Decsion tree using wrapper reverse
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFECV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Load the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Convert to DataFrame for ease of manipulation
df = pd.DataFrame(X, columns=iris.feature_names)
df['target'] = y

# Preprocessing: Scaling the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Define the Decision Tree classifier
decision_tree = DecisionTreeClassifier()

# Initialize RFECV
rfecv = RFECV(estimator=decision_tree, cv=StratifiedKFold(5), scoring='accuracy')

# Fit RFECV
rfecv.fit(X_scaled, y)

# Get selected features
selected_features = df.columns[rfecv.support_]

# Train-test split for detailed evaluation
X_selected = df[selected_features].values
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.3, random_state=42, stratify=y)

# Train the model using selected features
decision_tree.fit(X_train, y_train)

# Predict on test data
y_pred = decision_tree.predict(X_test)

# Performance measures
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Print performance measures
print(f'Accuracy on Test Data: {accuracy:.2f}')
print('Confusion Matrix:')
print(conf_matrix)
print('Classification Report:')
print(class_report)

# Confusion matrix heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=iris.target_names, yticklabels=iris.target_names)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# Plot number of features vs. cross-validation scores
plt.figure()
plt.title('Number of features vs. Cross-Validation scores')
plt.xlabel('Number of features selected')
plt.ylabel('Cross Validation Score (Accuracy)')
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
plt.show()


In [ ]:
# NN with wrapper reverse
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFECV
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Load the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Convert to DataFrame for ease of manipulation
df = pd.DataFrame(X, columns=iris.feature_names)
df['target'] = y

# Preprocessing: Scaling the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Define the neural network classifier
mlp = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=1000, random_state=42)

# Initialize RFECV
rfecv = RFECV(estimator=mlp, cv=StratifiedKFold(5), scoring='accuracy')

# Fit RFECV
rfecv.fit(X_scaled, y)

# Get selected features
selected_features = df.columns[rfecv.support_]

# Train-test split for detailed evaluation
X_selected = df[selected_features].values
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.3, random_state=42, stratify=y)

# Train the model using selected features
mlp.fit(X_train, y_train)

# Predict on test data
y_pred = mlp.predict(X_test)

# Performance measures
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Print performance measures
print(f'Accuracy on Test Data: {accuracy:.2f}')
print('Confusion Matrix:')
print(conf_matrix)
print('Classification Report:')
print(class_report)

# Confusion matrix heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=iris.target_names, yticklabels=iris.target_names)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# Plot number of features vs. cross-validation scores
plt.figure()
plt.title('Number of features vs. Cross-Validation scores')
plt.xlabel('Number of features selected')
plt.ylabel('Cross Validation Score (Accuracy)')
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
plt.show()


In [ ]:
# Logistic with wrapper reverse
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Load the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Convert to DataFrame for ease of manipulation
df = pd.DataFrame(X, columns=iris.feature_names)
df['target'] = y

# Preprocessing: Scaling the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Define the logistic regression classifier
log_reg = LogisticRegression(max_iter=1000, random_state=42)

# Initialize RFECV
rfecv = RFECV(estimator=log_reg, cv=StratifiedKFold(5), scoring='accuracy')

# Fit RFECV
rfecv.fit(X_scaled, y)

# Get selected features
selected_features = df.columns[rfecv.support_]

# Train-test split for detailed evaluation
X_selected = df[selected_features].values
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.3, random_state=42, stratify=y)

# Train the model using selected features
log_reg.fit(X_train, y_train)

# Predict on test data
y_pred = log_reg.predict(X_test)

# Performance measures
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Print performance measures
print(f'Accuracy on Test Data: {accuracy:.2f}')
print('Confusion Matrix:')
print(conf_matrix)
print('Classification Report:')
print(class_report)

# Confusion matrix heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=iris.target_names, yticklabels=iris.target_names)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# Plot number of features vs. cross-validation scores
plt.figure()
plt.title('Number of features vs. Cross-Validation scores')
plt.xlabel('Number of features selected')
plt.ylabel('Cross Validation Score (Accuracy)')
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
plt.show()


In [ ]:
# Knn with wrapper reverse
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFECV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Load the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Convert to DataFrame for ease of manipulation
df = pd.DataFrame(X, columns=iris.feature_names)
df['target'] = y

# Preprocessing: Scaling the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Define the KNN classifier
knn = KNeighborsClassifier()

# Initialize RFECV
rfecv = RFECV(estimator=knn, cv=StratifiedKFold(5), scoring='accuracy')

# Fit RFECV
rfecv.fit(X_scaled, y)

# Get selected features
selected_features = df.columns[rfecv.support_]

# Train-test split for detailed evaluation
X_selected = df[selected_features].values
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.3, random_state=42, stratify=y)

# Train the model using selected features
knn.fit(X_train, y_train)

# Predict on test data
y_pred = knn.predict(X_test)

# Performance measures
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Print performance measures
print(f'Accuracy on Test Data: {accuracy:.2f}')
print('Confusion Matrix:')
print(conf_matrix)
print('Classification Report:')
print(class_report)

# Confusion matrix heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=iris.target_names, yticklabels=iris.target_names)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# Plot number of features vs. cross-validation scores
plt.figure()
plt.title('Number of features vs. Cross-Validation scores')
plt.xlabel('Number of features selected')
plt.ylabel('Cross Validation Score (Accuracy)')
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
plt.show()


In [ ]:
# NB with wrapper reverse
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFECV
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Load the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Convert to DataFrame for ease of manipulation
df = pd.DataFrame(X, columns=iris.feature_names)
df['target'] = y

# Preprocessing: Scaling the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Define the Gaussian Naive Bayes classifier
gnb = GaussianNB()

# Initialize RFECV
rfecv = RFECV(estimator=gnb, cv=StratifiedKFold(5), scoring='accuracy')

# Fit RFECV
rfecv.fit(X_scaled, y)

# Get selected features
selected_features = df.columns[rfecv.support_]

# Train-test split for detailed evaluation
X_selected = df[selected_features].values
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.3, random_state=42, stratify=y)

# Train the model using selected features
gnb.fit(X_train, y_train)

# Predict on test data
y_pred = gnb.predict(X_test)

# Performance measures
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Print performance measures
print(f'Accuracy on Test Data: {accuracy:.2f}')
print('Confusion Matrix:')
print(conf_matrix)
print('Classification Report:')
print(class_report)

# Confusion matrix heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=iris.target_names, yticklabels=iris.target_names)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# Plot number of features vs. cross-validation scores
plt.figure()
plt.title('Number of features vs. Cross-Validation scores')
plt.xlabel('Number of features selected')
plt.ylabel('Cross Validation Score (Accuracy)')
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
plt.show()


In [ ]:
# Logistic OVR
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Load the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Convert to DataFrame for ease of manipulation
df = pd.DataFrame(X, columns=iris.feature_names)
df['target'] = y

# Preprocessing: Scaling the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split for detailed evaluation
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42, stratify=y)

# Define the logistic regression classifier
log_reg = LogisticRegression(max_iter=1000, random_state=42)

# Initialize One-vs-Rest Classifier
ovr_classifier = OneVsRestClassifier(log_reg)

# Train the model
ovr_classifier.fit(X_train, y_train)

# Predict on test data
y_pred = ovr_classifier.predict(X_test)

# Performance measures
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Print performance measures
print(f'Accuracy on Test Data: {accuracy:.2f}')
print('Confusion Matrix:')
print(conf_matrix)
print('Classification Report:')
print(class_report)

# Confusion matrix heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=iris.target_names, yticklabels=iris.target_names)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
# Using OvO Logistic
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsOneClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Load the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Convert to DataFrame for ease of manipulation
df = pd.DataFrame(X, columns=iris.feature_names)
df['target'] = y

# Preprocessing: Scaling the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split for detailed evaluation
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42, stratify=y)

# Define the logistic regression classifier
log_reg = LogisticRegression(max_iter=1000, random_state=42)

# Initialize One-vs-One Classifier
ovo_classifier = OneVsOneClassifier(log_reg)

# Train the model
ovo_classifier.fit(X_train, y_train)

# Predict on test data
y_pred = ovo_classifier.predict(X_test)

# Performance measures
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Print performance measures
print(f'Accuracy on Test Data: {accuracy:.2f}')
print('Confusion Matrix:')
print(conf_matrix)
print('Classification Report:')
print(class_report)

# Confusion matrix heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=iris.target_names, yticklabels=iris.target_names)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
# KNN regression with filter
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

# Load the dataset (example: Iris dataset for demonstration)
iris = load_iris()
X = iris.data
y = iris.target

# Convert to DataFrame for ease of manipulation
df = pd.DataFrame(X, columns=iris.feature_names)
df['target'] = y

# Preprocessing: Scaling the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Filter feature selection using linear coefficient
selector = SelectKBest(score_func=f_regression, k=3)  # Select top 3 features
X_selected = selector.fit_transform(X_scaled, y)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.3, random_state=42)

# KNN Regression with Grid Search for finding the best K
param_grid = {'n_neighbors': np.arange(1, 11)}  # Define different values of K
knn_reg = KNeighborsRegressor()
grid_search = GridSearchCV(knn_reg, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# Get the best K value
best_k = grid_search.best_params_['n_neighbors']

# Train KNN Regression model with the best K
knn_reg_best = KNeighborsRegressor(n_neighbors=best_k)
knn_reg_best.fit(X_train, y_train)

# Predict on the test set
y_pred = knn_reg_best.predict(X_test)

# Calculate performance measures
mse = mean_squared_error(y_test, y_pred)

print(f'Best K value: {best_k}')
print(f'Mean Squared Error on Test Data: {mse:.2f}')

# Visualize comparison graph
plt.figure(figsize=(10, 5))
plt.plot(y_test, label='Actual')
plt.plot(y_pred, label='Predicted')
plt.xlabel('Index')
plt.ylabel('Target')
plt.title('Comparison of Actual vs. Predicted')
plt.legend()
plt.show()


In [ ]:
# Knn regression with Wrapper
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

# Load the Boston housing dataset
boston = load_boston()
X = boston.data
y = boston.target

# Function to evaluate model performance with forward feature selection and KNN regressor
def evaluate_forward_selection_regression(X, y, reg, min_features, max_features, k_range):
    results = []
    for k in range(min_features, max_features + 1):
        sfs = SequentialFeatureSelector(reg, n_features_to_select=k, direction="forward", cv=KFold(n_splits=10, shuffle=True), scoring='neg_mean_squared_error')
        pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('selector', sfs),
            ('regressor', reg)
        ])
        param_grid = {'regressor__n_neighbors': k_range}
        grid_search = GridSearchCV(pipeline, param_grid, cv=KFold(n_splits=10, shuffle=True), scoring='neg_mean_squared_error')
        grid_search.fit(X, y)
        best_k = grid_search.best_params_['regressor__n_neighbors']
        cv_scores = cross_val_score(grid_search.best_estimator_, X, y, cv=KFold(n_splits=10, shuffle=True), scoring='r2')
        results.append((k, best_k, np.mean(cv_scores)))
    return results

# Evaluate forward feature selection and KNN regression
min_features = 1
max_features = X.shape[1]
k_range = list(range(1, 21))  # Define different values of K
results = evaluate_forward_selection_regression(X, y, KNeighborsRegressor(), min_features, max_features, k_range)

# Find the optimal number of features and K value
optimal_num_features, optimal_k = max(results, key=lambda x: x[2])[:2]
print(f'Optimal number of features: {optimal_num_features}')
print(f'Optimal K value: {optimal_k}')

# Plot the results
plt.figure(figsize=(10, 5))
plt.plot([r[0] for r in results], [r[2] for r in results], marker='o', linestyle='-', color='b')
plt.xlabel('Number of features selected')
plt.ylabel('Cross-validation R^2 score')
plt.title('Forward Feature Selection: Number of features selected vs. Cross-validation R^2 score')
plt.grid()
plt.show()

# Train-test split for detailed evaluation using the optimal number of features and K value
sfs_optimal = SequentialFeatureSelector(KNeighborsRegressor(n_neighbors=optimal_k), n_features_to_select=optimal_num_features, direction="forward", cv=KFold(n_splits=10, shuffle=True), scoring='neg_mean_squared_error')
pipeline_optimal = Pipeline([
    ('scaler', StandardScaler()),
    ('selector', sfs_optimal),
    ('regressor', KNeighborsRegressor(n_neighbors=optimal_k))
])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Fit the pipeline on training data
pipeline_optimal.fit(X_train, y_train)

# Predict on test data
y_pred = pipeline_optimal.predict(X_test)

# Performance measures for regression
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print performance measures for regression
print(f'Mean Squared Error on Test Data: {mse:.2f}')
print(f'R^2 Score on Test Data: {r2:.2f}')

# Plot the results
plt.figure(figsize=(10, 5))
plt.plot([r[0] for r in results], [r[2] for r in results], marker='o', linestyle='-', color='b')
plt.xlabel('Number of features selected')
plt.ylabel('Cross-validation R^2 score')
plt.title('Forward Feature Selection: Number of features selected vs. Cross-validation R^2 score')
plt.grid()
plt.show()

In [ ]:
# KNN regression with pca
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

# Load the Boston housing dataset
boston = load_boston()
X = boston.data
y = boston.target

# Function to evaluate model performance with PCA and KNN regressor
def evaluate_pca_regression(X, y, reg, min_components, max_components, k_range):
    results = []
    for k in range(min_components, max_components + 1):
        pca = PCA(n_components=k)
        pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('pca', pca),
            ('regressor', reg)
        ])
        param_grid = {'regressor__n_neighbors': k_range}
        grid_search = GridSearchCV(pipeline, param_grid, cv=KFold(n_splits=10, shuffle=True), scoring='neg_mean_squared_error')
        grid_search.fit(X, y)
        best_k = grid_search.best_params_['regressor__n_neighbors']
        cv_scores = cross_val_score(grid_search.best_estimator_, X, y, cv=KFold(n_splits=10, shuffle=True), scoring='r2')
        results.append((k, best_k, np.mean(cv_scores)))
    return results

# Evaluate PCA and KNN regression
min_components = 1
max_components = X.shape[1]
k_range = list(range(1, 21))  # Define different values of K
results = evaluate_pca_regression(X, y, KNeighborsRegressor(), min_components, max_components, k_range)

# Find the optimal number of components and K value
optimal_num_components, optimal_k = max(results, key=lambda x: x[2])[:2]
print(f'Optimal number of components: {optimal_num_components}')
print(f'Optimal K value: {optimal_k}')

# Plot the results
plt.figure(figsize=(10, 5))
plt.plot([r[0] for r in results], [r[2] for r in results], marker='o', linestyle='-', color='b')
plt.xlabel('Number of components selected')
plt.ylabel('Cross-validation R^2 score')
plt.title('PCA: Number of components selected vs. Cross-validation R^2 score')
plt.grid()
plt.show()

# Train-test split for detailed evaluation using the optimal number of components and K value
pca_optimal = PCA(n_components=optimal_num_components)
pipeline_optimal = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', pca_optimal),
    ('regressor', KNeighborsRegressor(n_neighbors=optimal_k))
])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Fit the pipeline on training data
pipeline_optimal.fit(X_train, y_train)

# Predict on test data
y_pred = pipeline_optimal.predict(X_test)

# Performance measures for regression
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print performance measures for regression
print(f'Mean Squared Error on Test Data: {mse:.2f}')
print(f'R^2 Score on Test Data: {r2:.2f}')

# Plot the results
plt.figure(figsize=(10, 5))
plt.plot([r[0] for r in results], [r[2] for r in results], marker='o', linestyle='-', color='b')
plt.xlabel('Number of components selected')
plt.ylabel('Cross-validation R^2 score')
plt.title('PCA: Number of components selected vs. Cross-validation R^2 score')
plt.grid()
plt.show()

In [ ]:
# KNN regression with svd
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

# Load the Boston housing dataset
boston = load_boston()
X = boston.data
y = boston.target

# Function to evaluate model performance with SVD and KNN regressor
def evaluate_svd_regression(X, y, reg, min_components, max_components, k_range):
    results = []
    for k in range(min_components, max_components + 1):
        svd = TruncatedSVD(n_components=k)
        pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('svd', svd),
            ('regressor', reg)
        ])
        param_grid = {'regressor__n_neighbors': k_range}
        grid_search = GridSearchCV(pipeline, param_grid, cv=KFold(n_splits=10, shuffle=True), scoring='neg_mean_squared_error')
        grid_search.fit(X, y)
        best_k = grid_search.best_params_['regressor__n_neighbors']
        cv_scores = cross_val_score(grid_search.best_estimator_, X, y, cv=KFold(n_splits=10, shuffle=True), scoring='r2')
        results.append((k, best_k, np.mean(cv_scores)))
    return results

# Evaluate SVD and KNN regression
min_components = 1
max_components = X.shape[1]
k_range = list(range(1, 21))  # Define different values of K
results = evaluate_svd_regression(X, y, KNeighborsRegressor(), min_components, max_components, k_range)

# Find the optimal number of components and K value
optimal_num_components, optimal_k = max(results, key=lambda x: x[2])[:2]
print(f'Optimal number of components: {optimal_num_components}')
print(f'Optimal K value: {optimal_k}')

# Plot the results
plt.figure(figsize=(10, 5))
plt.plot([r[0] for r in results], [r[2] for r in results], marker='o', linestyle='-', color='b')
plt.xlabel('Number of components selected')
plt.ylabel('Cross-validation R^2 score')
plt.title('SVD: Number of components selected vs. Cross-validation R^2 score')
plt.grid()
plt.show()

# Train-test split for detailed evaluation using the optimal number of components and K value
svd_optimal = TruncatedSVD(n_components=optimal_num_components)
pipeline_optimal = Pipeline([
    ('scaler', StandardScaler()),
    ('svd', svd_optimal),
    ('regressor', KNeighborsRegressor(n_neighbors=optimal_k))
])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Fit the pipeline on training data
pipeline_optimal.fit(X_train, y_train)

# Predict on test data
y_pred = pipeline_optimal.predict(X_test)

# Performance measures for regression
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print performance measures for regression
print(f'Mean Squared Error on Test Data: {mse:.2f}')
print(f'R^2 Score on Test Data: {r2:.2f}')

# Plot the results
plt.figure(figsize=(10, 5))
plt.plot([r[0] for r in results], [r[2] for r in results], marker='o', linestyle='-', color='b')
plt.xlabel('Number of components selected')
plt.ylabel('Cross-validation R^2 score')
plt.title('SVD: Number of components selected vs. Cross-validation R^2 score')
plt.grid()
plt.show()


In [ ]:
# Linear Regression with Filter
# Linear Regression with Filter Feature Selection
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Load the dataset (example: Iris dataset for demonstration)
iris = load_iris()
X = iris.data
y = iris.target

# Convert to DataFrame for ease of manipulation
df = pd.DataFrame(X, columns=iris.feature_names)
df['target'] = y

# Preprocessing: Scaling the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Filter feature selection using linear coefficient
selector = SelectKBest(score_func=f_regression, k=3)  # Select top 3 features
X_selected = selector.fit_transform(X_scaled, y)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.3, random_state=42)

# Linear Regression
linear_reg = LinearRegression()
linear_reg.fit(X_train, y_train)

# Predict on the test set
y_pred = linear_reg.predict(X_test)

# Calculate performance measures
mse = mean_squared_error(y_test, y_pred)

print(f'Mean Squared Error on Test Data: {mse:.2f}')

# Visualize comparison graph
plt.figure(figsize=(10, 5))
plt.plot(y_test, label='Actual')
plt.plot(y_pred, label='Predicted')
plt.xlabel('Index')
plt.ylabel('Target')
plt.title('Comparison of Actual vs. Predicted (Linear Regression)')
plt.legend()
plt.show()


In [ ]:
# Linear regression with wrapper
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

# Load the Boston housing dataset
boston = load_boston()
X = boston.data
y = boston.target

# Function to evaluate model performance with forward feature selection and linear regression
def evaluate_forward_selection_regression(X, y, min_features, max_features):
    results = []
    for k in range(min_features, max_features + 1):
        sfs = SequentialFeatureSelector(LinearRegression(), n_features_to_select=k, direction="forward", cv=KFold(n_splits=10, shuffle=True), scoring='neg_mean_squared_error')
        pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('selector', sfs),
            ('regressor', LinearRegression())
        ])
        pipeline.fit(X, y)
        cv_scores = cross_val_score(pipeline, X, y, cv=KFold(n_splits=10, shuffle=True), scoring='r2')
        results.append((k, np.mean(cv_scores)))
    return results

# Evaluate forward feature selection and linear regression
min_features = 1
max_features = X.shape[1]
results = evaluate_forward_selection_regression(X, y, min_features, max_features)

# Find the optimal number of features
optimal_num_features = max(results, key=lambda x: x[1])[0]
print(f'Optimal number of features: {optimal_num_features}')

# Plot the results
plt.figure(figsize=(10, 5))
plt.plot([r[0] for r in results], [r[1] for r in results], marker='o', linestyle='-', color='b')
plt.xlabel('Number of features selected')
plt.ylabel('Cross-validation R^2 score')
plt.title('Forward Feature Selection: Number of features selected vs. Cross-validation R^2 score')
plt.grid()
plt.show()

# Train-test split for detailed evaluation using the optimal number of features
sfs_optimal = SequentialFeatureSelector(LinearRegression(), n_features_to_select=optimal_num_features, direction="forward", cv=KFold(n_splits=10, shuffle=True), scoring='neg_mean_squared_error')
pipeline_optimal = Pipeline([
    ('scaler', StandardScaler()),
    ('selector', sfs_optimal),
    ('regressor', LinearRegression())
])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Fit the pipeline on training data
pipeline_optimal.fit(X_train, y_train)

# Predict on test data
y_pred = pipeline_optimal.predict(X_test)

# Performance measures for regression
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print performance measures for regression
print(f'Mean Squared Error on Test Data: {mse:.2f}')
print(f'R^2 Score on Test Data: {r2:.2f}')

# Plot the results
plt.figure(figsize=(10, 5))
plt.plot([r[0] for r in results], [r[2] for r in results], marker='o', linestyle='-', color='b')
plt.xlabel('Number of features selected')
plt.ylabel('Cross-validation R^2 score')
plt.title('Forward Feature Selection: Number of features selected vs. Cross-validation R^2 score')
plt.grid()
plt.show()

In [ ]:
# Linear regression with pca
# Linear Regression with PCA
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

# Load the Boston housing dataset
boston = load_boston()
X = boston.data
y = boston.target

# Function to evaluate model performance with PCA and Linear Regression
def evaluate_pca_regression(X, y, reg, min_components, max_components):
    results = []
    for k in range(min_components, max_components + 1):
        pca = PCA(n_components=k)
        pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('pca', pca),
            ('regressor', reg)
        ])
        cv_scores = cross_val_score(pipeline, X, y, cv=KFold(n_splits=10, shuffle=True), scoring='r2')
        results.append((k, np.mean(cv_scores)))
    return results

# Evaluate PCA and Linear Regression
min_components = 1
max_components = X.shape[1]
results = evaluate_pca_regression(X, y, LinearRegression(), min_components, max_components)

# Find the optimal number of components
optimal_num_components = max(results, key=lambda x: x[1])[0]
print(f'Optimal number of components: {optimal_num_components}')

# Plot the results
plt.figure(figsize=(10, 5))
plt.plot([r[0] for r in results], [r[1] for r in results], marker='o', linestyle='-', color='b')
plt.xlabel('Number of components selected')
plt.ylabel('Cross-validation R^2 score')
plt.title('PCA: Number of components selected vs. Cross-validation R^2 score')
plt.grid()
plt.show()

# Train-test split for detailed evaluation using the optimal number of components
pca_optimal = PCA(n_components=optimal_num_components)
pipeline_optimal = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', pca_optimal),
    ('regressor', LinearRegression())
])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Fit the pipeline on training data
pipeline_optimal.fit(X_train, y_train)

# Predict on test data
y_pred = pipeline_optimal.predict(X_test)

# Performance measures for regression
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print performance measures for regression
print(f'Mean Squared Error on Test Data: {mse:.2f}')
print(f'R^2 Score on Test Data: {r2:.2f}')

# Plot the results
plt.figure(figsize=(10, 5))
plt.plot([r[0] for r in results], [r[1] for r in results], marker='o', linestyle='-', color='b')
plt.xlabel('Number of components selected')
plt.ylabel('Cross-validation R^2 score')
plt.title('PCA: Number of components selected vs. Cross-validation R^2 score')
plt.grid()
plt.show()


In [ ]:
# Linear regression with svd
# Linear Regression with SVD
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

# Load the Boston housing dataset
boston = load_boston()
X = boston.data
y = boston.target

# Function to evaluate model performance with SVD and Linear Regression
def evaluate_svd_regression(X, y, reg, min_components, max_components):
    results = []
    for k in range(min_components, max_components + 1):
        svd = TruncatedSVD(n_components=k)
        pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('svd', svd),
            ('regressor', reg)
        ])
        cv_scores = cross_val_score(pipeline, X, y, cv=KFold(n_splits=10, shuffle=True), scoring='r2')
        results.append((k, np.mean(cv_scores)))
    return results

# Evaluate SVD and Linear Regression
min_components = 1
max_components = X.shape[1]
results = evaluate_svd_regression(X, y, LinearRegression(), min_components, max_components)

# Find the optimal number of components
optimal_num_components = max(results, key=lambda x: x[1])[0]
print(f'Optimal number of components: {optimal_num_components}')

# Plot the results
plt.figure(figsize=(10, 5))
plt.plot([r[0] for r in results], [r[1] for r in results], marker='o', linestyle='-', color='b')
plt.xlabel('Number of components selected')
plt.ylabel('Cross-validation R^2 score')
plt.title('SVD: Number of components selected vs. Cross-validation R^2 score')
plt.grid()
plt.show()

# Train-test split for detailed evaluation using the optimal number of components
svd_optimal = TruncatedSVD(n_components=optimal_num_components)
pipeline_optimal = Pipeline([
    ('scaler', StandardScaler()),
    ('svd', svd_optimal),
    ('regressor', LinearRegression())
])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Fit the pipeline on training data
pipeline_optimal.fit(X_train, y_train)

# Predict on test data
y_pred = pipeline_optimal.predict(X_test)

# Performance measures for regression
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print performance measures for regression
print(f'Mean Squared Error on Test Data: {mse:.2f}')
print(f'R^2 Score on Test Data: {r2:.2f}')

# Plot the results
plt.figure(figsize=(10, 5))
plt.plot([r[0] for r in results], [r[1] for r in results], marker='o', linestyle='-', color='b')
plt.xlabel('Number of components selected')
plt.ylabel('Cross-validation R^2 score')
plt.title('SVD: Number of components selected vs. Cross-validation R^2 score')
plt.grid()
plt.show()


In [ ]:
# KNN with wrapper reverse
# KNN Regression with RFECV
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFECV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

# Load the Boston housing dataset
boston = load_boston()
X = boston.data
y = boston.target

# Function to evaluate model performance with RFECV and KNN regressor
def evaluate_rfecv_regression(X, y, reg, k_range):
    rfecv = RFECV(estimator=reg, step=1, cv=KFold(n_splits=10, shuffle=True), scoring='neg_mean_squared_error')
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('rfecv', rfecv),
        ('regressor', reg)
    ])
    param_grid = {'regressor__n_neighbors': k_range}
    grid_search = GridSearchCV(pipeline, param_grid, cv=KFold(n_splits=10, shuffle=True), scoring='neg_mean_squared_error')
    grid_search.fit(X, y)
    best_k = grid_search.best_params_['regressor__n_neighbors']
    cv_scores = cross_val_score(grid_search.best_estimator_, X, y, cv=KFold(n_splits=10, shuffle=True), scoring='r2')
    return best_k, np.mean(cv_scores)

# Evaluate RFECV and KNN regression
k_range = list(range(1, 21))  # Define different values of K
optimal_k, mean_cv_score = evaluate_rfecv_regression(X, y, KNeighborsRegressor(), k_range)

# Print the optimal K value and mean cross-validation R^2 score
print(f'Optimal K value: {optimal_k}')
print(f'Mean Cross-validation R^2 score: {mean_cv_score:.2f}')

# Train-test split for detailed evaluation using the optimal K value
rfecv = RFECV(estimator=KNeighborsRegressor(n_neighbors=optimal_k), step=1, cv=KFold(n_splits=10, shuffle=True), scoring='neg_mean_squared_error')
pipeline_optimal = Pipeline([
    ('scaler', StandardScaler()),
    ('rfecv', rfecv),
    ('regressor', KNeighborsRegressor(n_neighbors=optimal_k))
])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Fit the pipeline on training data
pipeline_optimal.fit(X_train, y_train)

# Predict on test data
y_pred = pipeline_optimal.predict(X_test)

# Performance measures for regression
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print performance measures for regression
print(f'Mean Squared Error on Test Data: {mse:.2f}')
print(f'R^2 Score on Test Data: {r2:.2f}')

# Plot RFECV results
plt.figure(figsize=(10, 5))
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_, marker='o', linestyle='-', color='b')
plt.xlabel('Number of features selected')
plt.ylabel('Cross-validation score')
plt.title('RFECV: Number of features selected vs. Cross-validation score')
plt.grid()
plt.show()


In [ ]:
# Linear regression with wrapper reverse
# Linear Regression with RFECV
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import Pipeline

# Load the Boston housing dataset
boston = load_boston()
X = boston.data
y = boston.target

# Function to evaluate model performance with RFECV and Linear Regression
def evaluate_rfecv_regression(X, y):
    rfecv = RFECV(estimator=LinearRegression(), step=1, cv=KFold(n_splits=10, shuffle=True), scoring='neg_mean_squared_error')
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('rfecv', rfecv),
        ('regressor', LinearRegression())
    ])
    pipeline.fit(X, y)
    cv_scores = cross_val_score(pipeline, X, y, cv=KFold(n_splits=10, shuffle=True), scoring='r2')
    return rfecv.n_features_, np.mean(cv_scores)

# Evaluate RFECV and Linear Regression
optimal_num_features, mean_cv_score = evaluate_rfecv_regression(X, y)

# Print the optimal number of features and mean cross-validation R^2 score
print(f'Optimal number of features: {optimal_num_features}')
print(f'Mean Cross-validation R^2 score: {mean_cv_score:.2f}')

# Plot RFECV results
plt.figure(figsize=(10, 5))
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_, marker='o', linestyle='-', color='b')
plt.xlabel('Number of features selected')
plt.ylabel('Cross-validation score')
plt.title('RFECV: Number of features selected vs. Cross-validation score')
plt.grid()
plt.show()

# Train-test split for detailed evaluation using the optimal number of features
rfecv = RFECV(estimator=LinearRegression(), step=1, cv=KFold(n_splits=10, shuffle=True), scoring='neg_mean_squared_error')
pipeline_optimal = Pipeline([
    ('scaler', StandardScaler()),
    ('rfecv', rfecv),
    ('regressor', LinearRegression())
])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Fit the pipeline on training data
pipeline_optimal.fit(X_train, y_train)

# Predict on test data
y_pred = pipeline_optimal.predict(X_test)

# Performance measures for regression
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print performance measures for regression
print(f'Mean Squared Error on Test Data: {mse:.2f}')
print(f'R^2 Score on Test Data: {r2:.2f}')

# Plot RFECV results
plt.figure(figsize=(10, 5))
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_, marker='o', linestyle='-', color='b')
plt.xlabel('Number of features selected')
plt.ylabel('Cross-validation score')
plt.title('RFECV: Number of features selected vs. Cross-validation score')
plt.grid()
plt.show()


In [ ]:
import pandas as pd

# Load your own dataset
file_path = "your_dataset.csv"  # Replace "your_dataset.csv" with the path to your dataset file
data = pd.read_csv(file_path)

# Display the first few rows of the dataset
print(data.head())


In [ ]:
from sklearn.datasets import load_iris
from sklearn.datasets import load_boston
from sklearn.datasets import load_diabetes
from sklearn.datasets import load_digits
from sklearn.datasets import load_wine
from sklearn.datasets import load_breast_cancer
from sklearn.datasets import fetch_california_housing
from sklearn.datasets import load_linnerud


In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, Imputer

# Load dataset
iris = datasets.load_iris()
X = iris.data
y = iris.target

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocessing: Imputation of missing values
imputer = Imputer(strategy='mean')  # Impute missing values with mean
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

# Preprocessing: Feature scaling
scaler = StandardScaler()  # Standardize features by removing the mean and scaling to unit variance
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_test_scaled = scaler.transform(X_test_imputed)

# Now you can use X_train_scaled, X_test_scaled, y_train, y_test for further analysis or modeling


In [ ]:
from sklearn.datasets import load_iris

# Load the Iris dataset
iris = load_iris()

# Print the description of the dataset
print("Description:")
print(iris.DESCR)

# Print feature names
print("\nFeature names:")
print(iris.feature_names)

# Print target names
print("\nTarget names:")
print(iris.target_names)

# Print the shape of the data
print("\nData shape:")
print(iris.data.shape)

# Print the first few rows of the data
print("\nFirst few rows of the data:")
print(iris.data[:5])

# Print the target values
print("\nTarget values:")
print(iris.target)

import pandas as pd
from sklearn.datasets import load_iris

# Load the Iris dataset
iris = load_iris()

# Create a DataFrame from the dataset
iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)

# Print column information
print("Column Information:")
print(iris_df.info())

# Print summary statistics for each column
print("\nSummary Statistics:")
print(iris_df.describe())


In [ ]:
import pandas as pd

# Load your dataset into a DataFrame
# Replace 'your_dataset.csv' with the path to your dataset file
df = pd.read_csv('your_dataset.csv')

# Get the number of duplicate rows
duplicate_count = df.duplicated().sum()

# Get the count of null values in each column
null_count = df.isnull().sum()

print("Number of duplicate rows:", duplicate_count)
print("\nNull count in each column:")
print(null_count)
